## November 29 - Final class

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from scipy import optimize

## Classes

### HW5, Question 5: Regression and Curve fitting
*15 points*

In HW3, we looked at data from a compression test on a brittle material. Here, we are going to look at a test on a ductile strain-hardening material. The material deform's as per Hooke's law up to it's yield point, after which it follows a power-law hardening behavior.

The stress in the material is given by 

Elastic: $\sigma = E \epsilon_e \quad\quad\quad\quad\quad \sigma \leq \sigma_y$

Plastic: $\sigma = \sigma_y + \kappa \epsilon_p^n \quad\quad\quad \sigma > \sigma_y$

where:

$\sigma$ is the stress and \$\sigma_y$ is the yield stress;

$\epsilon_e$ and $\epsilon_p$ are the elastic and plastic strains, respectively;

$\kappa$ is the strength coefficient, and $n$ is the strain hardening exponent.

The compression test is peformed on a rectangular prismatic brittle specimen (cross-section area = 4x4 mm, length = 6 mm).

- Load the data from the provided `strain_hardening.csv` file using `pandas`. The data has already been cleaned for you. Displacement is in mm, force is in N.
- Perform a linear regression using `scipy.stats.linregress()` in the elastic regime of the data to calculate $E$ and $\sigma_y$.
- Perform a curve fit using `scipy.optimize.curve_fit()` in the plastic regime of the data to calculate $\kappa$ and $n$
- Print the values of all the variables calculated above using *scientific notation* with 2 decimal places. Use units of MPa for all parameters except $n$, which is dimensionless.
- Plot the raw stress (in MPa) vs strain curve, as well as the overall curve fit (elastic and plastic parts). The entire fit curve should be the same color. Mark the yield point. Make sure to include a legend.

**Note:** $\epsilon_p$ is the plastic strain, you must subtract the elastic strain ($\epsilon_e$) from the total strain to get the plastic strain.

Assume a 0.2% strain at the yield point; i.e., $\epsilon_y$ = 0.002. There is no plastic strain before yield and no elastic strain after yield.

In [ ]:
# Load data
csv_file = Path.cwd().parent / "Module 3 - Data Science with Python" / "strain_hardening.csv"
data = pd.read_csv(csv_file)
length = 6
area = 4*4

# Calculate strain and stress
data["strain"] = data["displacement"].div(length)
data["stress"] = data["force"].div(area)

# Plot
data.plot("strain", "stress", kind="scatter");

# Define yield strain
e_y = 0.002

In [ ]:
# Elastic regime
# Separate data before yield strain
data_elastic = data.copy()[data["strain"] <= e_y]

# Perform fit and get E
fit_result = stats.linregress(data_elastic["strain"], data_elastic["stress"])
E = fit_result.slope

# Calculate sigma_y and the regression line
sigma_y = E*e_y
fit_elastic = fit_result.slope*data_elastic["strain"] + fit_result.intercept

# Print
print(f"E = {E:.2e} MPa")
print(f"sigma_y = {sigma_y:.2e} MPa")

In [ ]:
from scipy import optimize

# Plastic regime
# Separate data after yield strain
data_plastic = data.copy()[data["strain"] > e_y]

# Define funtion for fit
def stress_func(strain, k, n):
    return sigma_y + k*(strain - e_y)**n

popt, pcov = optimize.curve_fit(stress_func,
                                data_plastic["strain"], data_plastic["stress"],
                                p0=(1000, 0.25))

# Calculate k, n and the fitted curve
k_opt, n_opt = popt
fit_plastic = stress_func(data_plastic["strain"], *popt)

print(f"K = {k_opt:.2e} MPa")
print(f"n = {n_opt:.2e}")

In [ ]:
# Plot raw data and fits
plt.plot(data["strain"], data["stress"], ".", label="Raw data");
plt.plot(data_elastic["strain"], fit_elastic, "r", label="Fit")
plt.plot(data_plastic["strain"], fit_plastic, "r")
plt.plot(e_y, sigma_y, "ok", label="Yield point")
plt.xlabel("Strain")
plt.ylabel("Stress (MPa)")
plt.legend(loc="lower right");

### With a class

In [ ]:
class StrainHardening:
    """
    Calculator for elastic modulus, yield strength, and strain hardening coefficients from tension/compression test data.

    Attributes
    ----------
    data : pandas DataFrame
        DataFrame containing the force (in N) and displacement (in mm).
    yield_strain : float
        Yield strain or elastic limit of the material
    """
    def __init__(self, data, yield_strain, length, area):
        self.data = data
        self.yield_strain = yield_strain
        self.length = length
        self.area = area
        
    def calculate_stress_strain(self):
        """Calculate stress and strain"""
        self.data["strain"] = self.data["displacement"].div(6)
        self.data["stress"] = self.data["force"].div(4*4)
        
        return self.data
    
    def calculate_elastic(self):
        """Calculate youngs modulus and yield strength (in MPa)."""
        
        # Separate data before yield strain
        self.data_elastic = self.data.copy()[self.data["strain"] <= self.yield_strain]

        # Perform fit and get E
        fit_result = stats.linregress(self.data_elastic["strain"], self.data_elastic["stress"])
        self.E = fit_result.slope

        # Calculate sigma_y and the regression line
        self.sigma_y = self.E*self.yield_strain
        fit_elastic = fit_result.slope*self.data_elastic["strain"] + fit_result.intercept

        # Print
        print(f"E = {self.E:.2e} MPa")
        print(f"sigma_y = {self.sigma_y:.2e} MPa")
        
        return self.E, self.sigma_y
    
    def calculate_plastic(self):
        """Calculate strain hardening parameters.""" 
        
        # Separate data after yield strain
        self.data_plastic = self.data.copy()[self.data["strain"] > self.yield_strain]

        # Define funtion for fit
        def stress_func(strain, k, n):
            return self.sigma_y + k*(strain - self.yield_strain)**n

        popt, pcov = optimize.curve_fit(stress_func,
                                        self.data_plastic["strain"], self.data_plastic["stress"],
                                        p0=(1000, 0.25))

        # Calculate k, n and the fitted curve
        self.k, self.n = popt
        fit_plastic = stress_func(self.data_plastic["strain"], *popt)

        print(f"K = {self.k:.2e} MPa")
        print(f"n = {self.n:.2e}")
        
        return self.k, self.n
    
    def calculate(self):
        """Perform elastic and plastic calculations"""
        
        self.calculate_stress_strain()
        E, sigma_y = self.calculate_elastic()
        k, n = self.calculate_plastic()
        
        return E, sigma_y, k, n
        
    def plot(self):
        """Plot data with elastic and plastic fits."""
        
        fig, ax = plt.subplots()
        ax.plot(self.data["strain"], self.data["stress"], ".", label="Raw data");
        ax.plot(self.data_elastic["strain"], fit_elastic, "r", label="Fit")
        ax.plot(self.data_plastic["strain"], fit_plastic, "r")
        ax.plot(self.yield_strain, self.sigma_y, "ok", label="Yield point")
        ax.set_xlabel("Strain")
        ax.set_ylabel("Stress (MPa)")
        ax.legend(loc="lower right");
        
        return fig

In [ ]:
# Load data
data = pd.read_csv("strain_hardening.csv")
yield_strain=0.002

# Instantiate an object
test_data = StrainHardening(data=data, yield_strain=yield_strain, length=6, area=4*4)

In [ ]:
# Use the convenient calculate method
E, sigma_y, k, n = test_data.calculate()

In [ ]:
E, sigma_y, = test_data.calculate_elastic()
E, sigma_y

In [ ]:
fig = test_data.plot()

## Python best practices and pythonic code

https://peps.python.org/pep-0008/

https://peps.python.org/pep-0020/

### Example 1 - avoid loops when possible

In [ ]:
# Generate data
data =  10*np.random.rand(500)

**Using loops:**

In [ ]:
%%timeit

processed_data = data**2 - 5*data

new_data = []
for value in processed_data:
    if value <= 0:
        new_value = value*-5
    else:
        new_value = value*2
    new_data.append(new_value)

**Using `map`:**

In [ ]:
def process_data(x):
    
    x = x**2 - 5*x

    if x <= 0:
        x_new = x*-5
    else:
        x_new = x*2
        
    return x_new

In [ ]:
%%timeit

new_data = map(process_data, data)

## Example 2 - loops with multiple iterators

In [ ]:
bars = ["The Bull", "Looseys", "Lebowskis", "Cry Babies", "The Top"]
drinks = ["wine", "rum and coke", "beer", "an Old Fashioned", "a Mule"]
prices = [7.50, 6.00, 4.50, 11.00, 8.50]

**No bueno:**

In [ ]:
for i in range(len(bars)):
    print(f"I drink {drinks[i]} at {bars[i]}. It costs ${prices[i]:.2f}.")

**Using `zip`:**

In [ ]:
for bar, drink, price in zip(bars, drinks, prices):
    print(f"I drink {drink} at {bar}. It costs ${price:.2f}.")

**With multiple items, you may be better off with a dictionary**

In [ ]:
salils_preferences = [
    {"bar": "The Bull", "drink": "wine", "price": 7.50},
    {"bar": "Looseys", "drink": "rum and coke", "price": 6.00},
    {"bar": "Lebowskis", "drink": "beer", "price": 4.50},
    {"bar": "Cry Babies", "drink": "an Old Fasioned", "price": 11.00},
    {"bar": "The Top", "drink": "a Mule", "price": 8.50},
]

for pref in salils_preferences:
    print(f"I drink {pref['drink']} at {pref['bar']}. It costs ${pref['price']:.2f}.")